<a href="https://colab.research.google.com/github/isnanramalia/cryptography2023/blob/main/seteganografi/Seteganografi_Image_%26_Audio_%2B_MSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note : - file gambar harus berekstensi .png
        - file audio harus berekstensi .wav

In [ ]:
#Prepare Library
!pip install pyfiglet
!pip install pydub
from PIL import Image
import wave
from pydub import AudioSegment
import numpy as np
from pyfiglet import Figlet
from subprocess import call,STDOUT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00


In [ ]:
#Image Function

# Convert encoding data ke 8-bit binary
# menggunakan nilai dari assci char
def genData(data):

        # list of binary codes
        # of given data
        newd = []

        for i in data:
            newd.append(format(ord(i), '08b'))
        return newd

# proses memodifikasi / menyisipkan pixel ke 8-bit binary cover
# data = text yang akan disisipkan
# pix = pixel dari cover gambar
# output_path = path untuk menyimpan file proses .txt
def modPix(pix, data,output_path):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    combined_text = f"Hidden text : {data} \nBinary text: {datalist}\n\nMengambil setiap 3 pixel pada cover sesuai panjang text :\n"

    with open(output_path, 'a') as file:
        file.write(combined_text)

    with open(output_path, 'a+') as file:
        c =1
        for i in range(lendata):
            # mengekstrak pixel cover gambar menjadi 3 pixel dalam waktu yang sama
            # hasil ekstrak akan menghasilkan 9 byte, 3 pixel x RGB(3 layer) = 9 block/byte
            pix = [value for value in imdata.__next__()[:3] +
                                    imdata.__next__()[:3] +
                                    imdata.__next__()[:3]]

            # menulis pixel hasil ekstrak ke proses.txt
            file.write(f"\nPixel {c} sampai Pixel {c+2}: {pix}")
            c+=3
            # Nilai Pixel harus sudah dibuat
            # proses penyisipan per 1 bit dari hidden text, ke per 1 byte dari cover
            file.write(f"\n Penyisipan dari {data[i]} = ")
            for j in range(0, 8):
                if (datalist[i][j] == '0' and pix[j] % 2 != 0):
                    pix[j] -= 1
                elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                    if(pix[j] != 0):
                        pix[j] -= 1
                    else:
                        pix[j] += 1
                file.write(f"{bin(pix[j])[2:]} ")


            # melakukan pembatasan proses jika sudah berada
            #  di char terakhir hidden text maka stop
            if (i == lendata - 1):
                if (pix[-1] % 2 == 0):
                    if(pix[-1] != 0):
                        pix[-1] -= 1
                    else:
                        pix[-1] += 1
            else:
                if (pix[-1] % 2 != 0):
                    pix[-1] -= 1

            pix = tuple(pix)

            # melakukan ekstraksi setiap 3 pixel
            yield pix[0:3]
            yield pix[3:6]
            yield pix[6:9]


# fungsi untuk meletakan pixel modifikasi ke gambar encode
def encode_enc(newimg, data, output_path):
    w = newimg.size[0]
    (x, y) = (0, 0)

    for pixel in modPix(newimg.getdata(), data, output_path):

        # meletakan pixel encode pada gambar baru
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1


# Encode data ke image
def encodeImage():
    img = input("Masukkan Path Gambar (Image.png) : ")
    image = Image.open(img, 'r')

    data = input("Masukkan Kata yang akan disembunyikan : ")
    if (len(data) == 0):
        raise ValueError('Kata tidak boleh kosong!')

    output_path = input("Masukkan Path Hasil Proses (proses.txt) : ")
    newimg = image.copy()
    encode_enc(newimg, data, output_path)

    new_img_name = input("Masukkan Path hasil Encode Gambar (Encode.png) : ")
    newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))

    binary_data  = iter(newimg.getdata())
    bin = [value for value in binary_data]
    with open(output_path, 'a+') as file:
        file.write(f"\nhasil encode : {bin}")

# Decode data dari image Decode
def decodeImage():
    img = input("Masukkan Path hasil Encode Gambar (Encode.png) : ")
    image = Image.open(img, 'r')

    output_path = input("Masukkan Path Hasil Proses (proses.txt) : ")

    combined_text = f"Menyiapkan Cover sesuai panjang text :\n"

    with open(output_path, 'a') as file:
        file.write(combined_text)

    data = ''
    imgdata = iter(image.getdata())
    with open(output_path, 'a+') as file:
      c = 1
      while (True):
          # melakukan ekstraksi setiap 3 pixel pada waktu yang sama
          pixels = [value for value in imgdata.__next__()[:3] +
                                  imgdata.__next__()[:3] +
                                  imgdata.__next__()[:3]]

          # menulis string dari binary pixel
          file.write(f"\nPixel {c} sampai Pixel {c+2}: {pixels}\n")
          binstr = ''

          # melakukan pengambilan binary dari setiap blok/byte
          for i in pixels[:8]:
              if (i % 2 == 0):
                  binstr += '0'
              else:
                  binstr += '1'
          file.write(f"Mengambil byte terakhir {c} sampai Pixel {c+2}: {binstr}\n")

          data += chr(int(binstr, 2))
          file.write(f"Pesan yang tersembunyi dari pixel 1 sampai {c+2} : {data}\n")
          c+=3
          #  pengembalian hidden text hasil decode
          if (pixels[-1] % 2 != 0):
              return data

In [ ]:
# Fungsi encode audio
# Fungsi konversi mp3 ke wav
def mp3_to_wav(mp3_path):
    audio = AudioSegment.from_mp3(mp3_path)
    wav_path = mp3_path.replace(".mp3", ".wav")
    audio.export(wav_path, format="wav")
    return wav_path

# fungsi untuk encode audio
def encodeAudio():
    print("=== Encode Audio ===")

    # Input path audio
    audioName = input("Masukkan Path Audio .wav (song.wav): ")

    # convert audio wav to mp3
    if audioName.lower().endswith('.mp3'):
        audioName = mp3_to_wav(audioName)

    # Input path untuk hasil encode dan plain text
    encodeAudioName = input("Masukkan Path Hasil Encode .wav (encode.wav): ")
    plainText = input("Masukkan Hidden Text: ")

    print("\nProses Encode:")

    # Membaca file audio
    song = wave.open(audioName, mode='rb')
    frame_bytes = bytearray(list(song.readframes(song.getnframes())))

    # Menambahkan teks ke dalam audio
    log_text = []
    log_text.append("Teks Awal: " + plainText)
    plainText = plainText + int((len(frame_bytes)-(len(plainText)*8*8))/8) *'#'
    bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8,'0') for i in plainText])))

    for i, bit in enumerate(bits):
        original_byte = frame_bytes[i]
        original_bits = [int(b) for b in f"{original_byte:08b}"]
        frame_bytes[i] = (frame_bytes[i] & 254) | bit
        modified_byte = frame_bytes[i]
        modified_bits = [int(b) for b in f"{modified_byte:08b}"]

        log_text.append(f"Frame {i+1}: Bit Awal {original_bits}, Bit Tersembunyi {bit}, Bit Sesudah {modified_bits} -> {frame_bytes[i]}")

    frame_modified = bytes(frame_bytes)

    # Menyimpan hasil encode
    with wave.open(encodeAudioName, 'wb') as fd:
        fd.setparams(song.getparams())
        fd.writeframes(frame_modified)

    song.close()

    # Menyimpan log ke dalam file teks
    with open("encode_log.txt", "w") as log_file:
        log_file.write("\n".join(log_text))

    print("Encode audio berhasil disimpan ke path", encodeAudioName)
    print("Log proses encode disimpan ke encode_log.txt")

# Fungsi decode audio
def decodeAudio():
    print("=== Decode Audio ===")

    # Input path encode audio
    audioName = input("Masukkan Path Encode Audio .wav (encode.wav): ")

    if audioName.lower().endswith('.mp3'):
        audioName = mp3_to_wav(audioName)

    print("\nProses Decode:")

    # Membaca file audio encode
    song = wave.open(audioName, mode='rb')
    frame_bytes = bytearray(list(song.readframes(song.getnframes())))

    # Mengambil bit tersembunyi
    log_text = []
    log_text.append("Mengambil Bit Tersembunyi:")
    extracted = [frame_bytes[i] & 1 for i in range(len(frame_bytes))]
    log_text.append("Bit Tersembunyi: " + "".join(map(str, extracted)))

    # Mengonversi bit ke teks
    string = "".join(chr(int("".join(map(str,extracted[i:i+8])),2)) for i in range(0,len(extracted),8))
    plainText = string.split("###")[0]

    log_text.append("\nHasil Decode:")
    log_text.append("Hidden Text: " + plainText)
    print("hidden text : ", plainText)
    song.close()

    # Menyimpan log ke dalam file teks
    with open("decode_log.txt", "w") as log_file:
        log_file.write("\n".join(log_text))

    print("Log proses decode disimpan ke decode_log.txt")

In [ ]:
#MSE Function

def calculate_mse_image(original_image_path, stegano_image_path):
    # membuka path file gambar
    original_image = Image.open(original_image_path)
    stegano_image = Image.open(stegano_image_path)

    # convert gambar ke numpy arrays
    original_array = np.array(original_image)
    stegano_array = np.array(stegano_image)

    # memastikan kedua gambar memiliki ukuran pixel yang sama
    if original_array.shape != stegano_array.shape:
        raise ValueError("Images must have the same dimensions.")

    # menghitung MSE (Mean Sequared Error)
    mse = np.mean((original_array - stegano_array) ** 2)
    return mse

# fungsi untuk load audio dari path
def load_wav(file_path):
    with wave.open(file_path, 'rb') as wf:
        framerate = wf.getframerate()
        frames = wf.readframes(wf.getnframes())
        audio_data = np.frombuffer(frames, dtype=np.int16)
    return audio_data, framerate

def load_mp3(file_path):
    audio = AudioSegment.from_mp3(file_path)
    return np.array(audio.get_array_of_samples()), audio.frame_rate

# fungsi penghitungan MSE audio
def calculate_mse_audio(original_audio_path, steganographic_audio_path):
    # membaca jenis file audio
    if original_audio_path.lower().endswith('.wav'):
        original_audio, framerate_original = load_wav(original_audio_path)
    elif original_audio_path.lower().endswith('.mp3'):
        original_audio, framerate_original = load_mp3(original_audio_path)
    else:
        raise ValueError("File format tidak didukung, silahkan pakai .mp3/wav")

    if steganographic_audio_path.lower().endswith('.wav'):
        steganographic_audio, framerate_steganographic = load_wav(steganographic_audio_path)
    elif steganographic_audio_path.lower().endswith('.mp3'):
        steganographic_audio, framerate_steganographic = load_mp3(steganographic_audio_path)
    else:
        raise ValueError("File format tidak didukung, silahkan pakai .mp3/wav")


     # memastikan file audio memiliki panjang yang sama
    min_length = min(len(original_audio), len(steganographic_audio))
    original_audio = original_audio[:min_length]
    steganographic_audio = steganographic_audio[:min_length]

    # menghitung MSE (Mean Sequared Error)
    mse = np.mean((original_audio - steganographic_audio) ** 2)
    return mse

In [ ]:
#Main Function
def main():
 while(1):
    f = Figlet(font='slant')
    print(f.renderText("Steganagraphy"))
    print("Menu :")
    print("(1) Steganografi Image")
    print("(2) Steganografi Audio")
    print("(3) Check MSE")
    print("(4) Exit")
    print("-----------------------")
    choice = input("(!) Pilih 1, 2, 3, 4 : ")
    if (choice == "1"):
      print("Pilih :")
      print("(a) Encode Image")
      print("(b) Decode Image")
      print("(c) Exit")
      print("-----------------------")
      choiceSection = input("(!) Pilih a, b, c : ")
      if (choiceSection == "a"):
        try:
          encodeImage()
        except:
          print("Pesan anda lebih dari besar dari batas karakter pixelXpixel ")
      elif (choiceSection == 'b'):
        print("hasil encode : ",decodeImage())
      elif (choiceSection == 'c'):
        print("keluar dari program, Semoga Harimu Menyenangkan :)")
        break
      else:
        print("pilih menu a, b, atau c")
    elif (choice == '2'):
      print("Pilih :")
      print("(a) Encode Audio")
      print("(b) Decode Audio")
      print("(c) Exit")
      print("-----------------------")
      choiceSection = input("(!) Pilih a, b, c : ")
      if (choiceSection == "a"):
        try:
          encodeAudio()
        except:
          print("Pesan anda lebih dari besar dari batas karakter frame ")
      elif (choiceSection == 'b'):
        decodeAudio()
      elif (choiceSection == 'c'):
        print("keluar dari program, Semoga Harimu Menyenangkan :)")
        break
      else:
        print("pilih menu a, b, atau c")
    elif (choice == '3'):
      print("Pilih :")
      print("(a) Check MSE Image")
      print("(b) Check MSE Audio")
      print("(c) Exit")
      print("-----------------------")
      choiceSection = input("(!) Pilih a, b, c : ")
      if (choiceSection == "a"):
        original_image_path = input("Masukkan Path Image Original/Cover (original.png) : ")
        stegano_image_path = input("Masukkan Path Image Encode (encode.png) : ")
        print(f"MSE Image anda : {calculate_mse_image(original_image_path, stegano_image_path)}")
      elif (choiceSection == 'b'):
        original_audio_path = input("Masukkan Path Audio Original/Cover (original.wav) :")
        steganographic_audio_path = input("Masukkan Path Audio Encode (encode.wav) : ")
        print(f"MSE Audio anda : {calculate_mse_audio(original_audio_path, steganographic_audio_path)}")
      elif (choiceSection == 'c'):
        print("keluar dari program, Semoga Harimu Menyenangkan :)")
        break
      else:
        print("pilih menu a, b, atau c")
    elif (choice == '4'):
        print("keluar dari program, Semoga Harimu Menyenangkan :)")
        break

if __name__ == "__main__":
    main()

   _____ __                                                      __         
  / ___// /____  ____ _____ _____  ____ _____ __________ _____  / /_  __  __
  \__ \/ __/ _ \/ __ `/ __ `/ __ \/ __ `/ __ `/ ___/ __ `/ __ \/ __ \/ / / /
 ___/ / /_/  __/ /_/ / /_/ / / / / /_/ / /_/ / /  / /_/ / /_/ / / / / /_/ / 
/____/\__/\___/\__, /\__,_/_/ /_/\__,_/\__, /_/   \__,_/ .___/_/ /_/\__, /  
              /____/                  /____/          /_/          /____/   

Menu :
(1) Steganografi Image
(2) Steganografi Audio
(3) Check MSE
(4) Exit
-----------------------
Pilih :
(a) Encode Image
(b) Decode Image
(c) Exit
-----------------------
   _____ __                                                      __         
  / ___// /____  ____ _____ _____  ____ _____ __________ _____  / /_  __  __
  \__ \/ __/ _ \/ __ `/ __ `/ __ \/ __ `/ __ `/ ___/ __ `/ __ \/ __ \/ / / /
 ___/ / /_/  __/ /_/ / /_/ / / / / /_/ / /_/ / /  / /_/ / /_/ / / / / /_/ / 
/____/\__/\___/\__, /\__,_/_/ /_/\__,_/\__, /_/   \__,